In [ ]:
# read data
import pandas as pd
# Set the maximum number of rows and columns to display
pd.set_option('display.max_rows', 200)  # Adjust this number as needed
pd.set_option('display.max_columns', 50)  # Adjust this number as needed

## 1. Assign CBGID

### Get shapefile of california

In [ ]:
# Get shapefiles
import geopandas as gpd
poly = gpd.GeoDataFrame.from_file('../../data_CityEvent/Shp/US_blck_grp_2019.shp')
poly = poly.to_crs(epsg=4326)
california = poly[poly['STATEFP']=='06']

### CBGID assign function + Handle different geometry: Point, Polygon, and MultiPolygon

In [ ]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import shape

def parse_geometry(geo):
    """Parse the GEO field into a Shapely geometry (Point, Polygon, or MultiPolygon)."""
    if pd.isna(geo):
        return None
    try:
        # Convert the GEO string into a Python dict
        geo_json = json.loads(geo)

        # Use shapely's shape() to parse the geometry
        geometry = shape(geo_json)

        # Ensure the parsed geometry is one of the expected types
        if geometry.geom_type in {"Point", "Polygon", "MultiPolygon"}:
            return geometry
        else:
            print(f"Unsupported geometry type: {geometry.geom_type}")
            return None
    except Exception as e:
        print(f"Error parsing geometry: {e}")
        return None

def assign_cbgid(df, shapefile):
    df['geometry'] = df['GEO'].apply(parse_geometry)
    df = df.dropna(subset=['geometry'])
    df = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
    shapefile = gpd.GeoDataFrame(shapefile, geometry='geometry', crs="EPSG:4326")
    shapefile = shapefile[['GEOID', 'geometry']]

    assigned_df = gpd.sjoin(df, shapefile, how='inner', predicate='intersects')
    print(df.shape)
    print(assigned_df.shape)
    assigned_df = assigned_df.drop(columns=['index_right'])
    print(assigned_df.shape)
    return(assigned_df)

### Save data by event category in california only

In [ ]:
# Save function
def save_df_by_event_category(df, event_type='event_type'):
    columns = [col for col in df.columns]
    categories = list(df['CATEGORY'].unique())
    for category in categories:
        df_by_category = df[df['CATEGORY'] == category]
        print(f'{category} shape: {df_by_category.shape}')
        df_by_category.to_csv(f'../../data_CityEvent/processed/1_cbgid_assigned_by_category/{event_type}{category}.csv', index=False)  # Set index=False to avoid saving the index
    
    print('Successful')

### Function for handling cancelled/postponed events data + CBGID assigner

In [ ]:
from utils.time.time_converted_to_boston import convert_time_of_dataframe

def dropping_rows_and_cbgid_assigner(df):
    df = df[(df['CANCELLED_DT'].isna()) & (df['POSTPONED_DT'].isna())]

    df = df.drop(['CANCELLED_DT', 'POSTPONED_DT'], axis=1)

    datetime_columns = ['CREATE_DT', 'UPDATE_DT', 'EVENT_START', 'EVENT_END', 'PREDICTED_END', 'ROW_INSERTED_DT', 'ROW_UPDATED_DT']
    df[datetime_columns] = df[datetime_columns].apply(pd.to_datetime, errors='coerce')

    df_converted = convert_time_of_dataframe(df)

    df_cbgid = assign_cbgid(df_converted, california)

    save_df_by_event_category(df_cbgid)


### Attended Events

In [ ]:
df_attended = pd.read_csv('../../data_CityEvent/Cityevents/Demand_Intelligence_for_Attended_Events_California-0.csv')

In [ ]:
# Save attended
dropping_rows_and_cbgid_assigner(df_attended, 'attended')

### Non-Attended Events

In [ ]:
df_non_attended = pd.read_csv('../../data_CityEvent/Cityevents/Demand_Intelligence_for_Non_Attended_Events_California-0.csv')
df_non_attended.shape

In [ ]:
# Save non-attended
dropping_rows_and_cbgid_assigner(df_non_attended, 'non_attended')

### Unscheduled Events

In [ ]:
df_unscheduled = pd.read_csv('../../data_CityEvent/Cityevents/Demand_Intelligence_for_Unscheduled_Events_North_America-0.csv')
df_unscheduled.shape

In [ ]:
# Save function
def save_df_by_event_category_intermediate(df):

    categories = list(df['CATEGORY'].unique())

    for category in categories:
        df_by_category = df[df['CATEGORY'] == category]
        print(f'{category} shape: {df_by_category.shape}')
        df_by_category.to_csv(f'../../data_CityEvent/processed/1_cbgid_assigned_by_category/unscheduled_intermediate_{category}.csv', index=False)  
        # Set index=False to avoid saving the index
    print('Successful')

In [ ]:
save_df_by_event_category_intermediate(df_unscheduled)

In [ ]:
import os
filepath = '../../data_CityEvent/processed/1_cbgid_assigned_by_category/unscheduled_intermediate'
files = os.listdir(filepath)
# print(files)

for file in files:
    df = pd.read_csv(f'../../data_CityEvent/processed/1_cbgid_assigned_by_category/unscheduled_intermediate/{file}')
    print(file)
    print('intermediate shape': df.shape)
    df_assigned = assign_cbgid(df, california)
    print('shape after assigning': df_assigned.shape)
    df_assigned.to_csv(f'../../data_CityEvent/processed/1_cbgid_assigned_by_category/{file}', index=False) 
    print(f'{file} Save Successful')   

### Downscale the CBGID assigned data

In [ ]:
folder_path = '../../data_CityEvent/processed/1_cbgid_assigned_by_category'
file_names = os.listdir(folder_path)
for file_name in file_names:
    print(file_name)

In [ ]:
# downscale
columns_to_keep = [
    'EVENT_ID',
    'CATEGORY',
    'GEO',
    'EVENT_START_BOSTON',
    'EVENT_END_BOSTON',
    'geometry',
    'GEOID'
]

for file_name in file_names:
    df = pd.read_csv(f'../../data_CityEvent/processed/1_cbgid_assigned_by_category/{file_name}')
    print(file_name, df.shape)
    df = df[columns_to_keep]
    print(file_name, df.shape)
    df.to_csv(f'../../data_CityEvent/processed/2_cbgid_downscaled/{file_name}', index=False)
    print('Saved')

## 2. 

### a

In [ ]:
folder_path = '../../data_CityEvent/processed/2_cbgid_downscaled/'
# List all files in the folder
file_names = os.listdir(folder_path)



In [ ]:
import pandas as pd

def hour_rows_df(df):
    # Convert the EVENT_START_BOSTON and EVENT_END_BOSTON to datetime
    datetime_columns = ['EVENT_START_BOSTON', 'EVENT_END_BOSTON']
    for datetime_column in datetime_columns:
        # Convert the specified columns to datetime format
        df[datetime_column] = pd.to_datetime(df[datetime_column], errors='coerce')

    columns_to_keep = ['EVENT_ID', 'GEOID', 'geometry', 'EVENT_START_BOSTON', 'EVENT_END_BOSTON', 'CATEGORY']
    df = df[columns_to_keep]

    # Define a function to create a range of hourly timestamps between start and end times
    def create_hourly_rows(row):
        start_time = row['EVENT_START_BOSTON'].floor('H')  # Round down to nearest hour
        end_time = row['EVENT_END_BOSTON'].ceil('H')  # Round up to nearest hour
        return pd.date_range(start=start_time, end=end_time, freq='H')

    # Apply the function to each row and store the result in a new column 'hourly_times'
    df['hourly_times'] = df.apply(create_hourly_rows, axis=1)

    # Explode the DataFrame to create one row per hour
    expanded_df = df.explode('hourly_times')

    return expanded_df


In [ ]:
for file_name in file_names:
    # if file_name[0] == 'n' or file_name[0] == 'u':
    #     continue
    current_df = pd.read_csv(f'{folder_path}/{file_name}')
    print(f'{file_name}: {current_df.shape}')
    datetime_columns = ['EVENT_START_BOSTON', 'EVENT_END_BOSTON']
    for datetime_column in datetime_columns:
        # Convert the specified columns to datetime format
        current_df[datetime_column] = pd.to_datetime(current_df[datetime_column], errors='coerce')
    
    # Drop rows with NaN (or NaT) in any of the datetime columns
    current_df.dropna(subset=datetime_columns, inplace=True)
    
    # Check the shape after dropping rows with NaT values
    print(f'{file_name} after dropping NaT: {current_df.shape}')

    current_df_hourly = hour_rows_df(current_df)    
    file_name = file_name[:-4]
    current_df_hourly.to_csv(f'../../data_CityEvent/processed/3_hourly_events_cbgid_category/{file_name}_hourly.csv', index=False)